# Copyright : fast.ai - Jeremy Howard & Sylvain Gugger - 2020 (GPLv3)

Cellules de code et plan du notebook adaptées du livre :

Deep Learning for Coders with fastai & PyTorch de Jeremy Howard et Sylvain Gugger.

The code in the original notebooks (and thus the code in this notebook) is covered by the GPL v3 license; see the LICENSE file for details.

### Dépendances à installer

In [1]:
#!pip install fastcore --upgrade
#!pip install fastai2 --upgrade
#!pip install sentencepiece==0.1.86

# NLP : Traitement du langage naturel - Recurrent Neural Networks

Nous allons entrainer un **language model** : un modèle capable de deviner le mot suivant dans un texte à partir des mots qui le précèdent. 

Ce type de tâche est appelé **self-supervised learning** : nous n'avons pas besoin de créer explicitement des labels pour entrainer ce type de modèle, il suffit de le nourrir de beaucoup, beaucoup de textes. Pour cette tâche, nous avons un processus pour obtenir automatiquement les labels à prédire à partir des données brutes.

Cette tâche n'est pas triviale : pour deviner correctement le mot suivant dans une phrase, le modèle devra développer une compréhension assez approfondie de la langue. 

Le **self-supervised learning** peut également être utilisé dans d'autres domaines, c'est un sujet de recherche très actif en ce moment, par exemple en vision : [Prototypical Contrastive Learning](https://blog.einstein.ai/prototypical-contrastive-learning-pushing-the-frontiers-of-unsupervised-learning/). 

Un modèle entrainé par **self-supervised learning** n'est en général pas utilisé directement : il sert de point de départ pour réaliser un fine-tuning en mode supervisé sur une beaucoup plus petite quantité de données.

Nous allons travailler sur le jeu de données [IMDb sentiment analysis](https://www.imdb.com/title/tt0107144/reviews?ref_=tt_ov_rt) :
- 50.000 critiques de films sans labels positifs ou négatifs
- 25 000 critiques avec des labels dans le jeu d'entrainement
- 25 000 critiques avec des labels dans le jeu de validation
- => 100 000 critiques de films au total

Nous pouvons utiliser toutes ces critiques pour affiner un language model pré-entrainé sur les articles de Wikipédia : cela permettra d'obtenir un language model particulièrement efficace pour prédire le prochain mot d'une critique de film.

Cette approche appelée [Universal Language Model Fine-tuning (ULMFit)](https://arxiv.org/abs/1801.06146) a été inventée lors du cours fastai 2017-2018. C'était le premier exemple de transfert d'apprentissage sur du texte. Des benchmarks sont régulièrement réalisés avec des méthodes plus modernes et beaucoup plus lourdes en termes de calcul : cette méthode reste un très bon compromis coût/performance pour la classification de textes.

Cette publication a montré que l'étape supplémentaire de fine tuning du language model, avant de transférer l'apprentissage à une tâche de classification, permettait d'obtenir des prévisions nettement meilleures.

En utilisant cette approche, nous avons trois étapes pour le transfert de l'apprentissage en NLP (Natural Language Processing) :
1. Language model - self-supervised - entrainement initial : Wikitext 103
2. Language mode - self-supervised - fine tuning : IMDb
3. Classifier - supervisé - fine tuning : IMDb

## Preprocessing du texte

Nous avons déjà vu comment les variables de type catégorie (énumération - liste de niveaux finie) peuvent être utilisées en entrée d'un réseau de neurones.

L'approche que nous avons adoptée était la suivante :
1. Faire une liste de toutes les valeurs possibles de cette catégorie (nous appellerons cette liste le vocabulaire).
2. Remplacer chaque valeur par son index dans le vocabulaire.
3. Créer une matrice d'embeddings contenant une représentation de chaque valeur (c'est-à-dire pour chaque élément du vocabulaire) sous forme d'une liste de nombres .
4. Utiliser cette matrice d'embeddings comme première couche d'un réseau de neurones. 

Les mots de vocabulaire d'une langue donnée peuvent être traités comme les différents niveaux d'une variable de type catégorie : on va retenir une liste finie de mots et **apprendre une représentation numérique sous forme d'embedding pour chacun des mots du vocabulaire retenu**.

Ce qui est nouveau, c'est l'idée d'une **séquence**. D'abord, nous concaténons tous les documents de notre jeu de données en une longue chaîne et nous la divisons en mots, ce qui nous donne une très longue liste de mots (ou "tokens"). La donnée d'entrée de notre modèle sera la séquence de mots commençant par le premier mot de notre très longue liste et se terminant par l'avant-dernier, et notre variable à prédire sera la séquence de mots commençant par le deuxième mot et se terminant par le dernier.

Notre vocabulaire sera constitué d'un mélange de mots communs qui sont déjà dans le vocabulaire de notre modèle pré-entrainé sur Wikipedia et de nouveaux mots spécifiques à notre corpus (termes cinématographiques ou noms d'acteurs, par exemple). Notre matrice d'embeddings sera construite en conséquence : pour les mots qui sont dans le vocabulaire de notre modèle pré-entrainé, nous prendrons la ligne correspondante dans la matrice d'embedding du modèle pré-entrainé ; mais pour les nouveaux mots, nous n'aurons rien, donc nous initialiserons juste la ligne correspondante avec un vecteur aléatoire.

Chacune des étapes nécessaires à la création d'un language model est associée à du jargon issu du monde du traitement du langage naturel, et des classes de fastai et PyTorch sont disponibles pour aider. Les étapes sont les suivantes :

- Tokenisation : Convertir le texte en une liste de mots (ou de caractères, ou de sous-chaînes de caractères, selon la granularité de votre modèle)
- Conversion en numéros : Faire une liste de tous les mots uniques qui apparaissent (le vocabulaire), et convertir chaque mot en un numéro, en recherchant son index dans le vocabulaire
- Création d'un DataLoader pour le language model : fastai fournit une classe LMDataLoader qui gère automatiquement la création d'une variable à prédire qui est décalée de la variable d'entrée d'un token. Elle gère également certains détails importants, tels que la manière de mélanger les données d'apprentissage de manière à ce que les variables dépendantes et indépendantes conservent leur structure
- Création d'un language model : Nous avons besoin d'un type de modèle spécial qui fait quelque chose que nous n'avons jamais vu auparavant : il gère des listes de saisie qui peuvent être arbitrairement grandes ou petites. Il existe plusieurs façons de le faire ; dans ce chapitre, nous utiliserons un réseau neuronal récurrent (RNN).

### Tokenisation

Lorsque nous avons parlé de "convertir le texte en une liste de mots", nous avons omis beaucoup de détails. Par exemple, que faisons-nous de la ponctuation ? Comment traiter un mot comme "don't" ? S'agit-il d'un seul mot ou de deux ? Qu'en est-il des longs mots médicaux ou chimiques ? Doivent-ils être séparés en parties distinctes ? Et les mots avec un trait d'union ? Qu'en est-il des langues comme l'allemand et le polonais, où nous pouvons créer des mots vraiment longs à partir de nombreux morceaux ? Qu'en est-il des langues comme le japonais et le chinois qui n'utilisent pas du tout d'espaces et qui n'ont pas vraiment une idée bien définie du mot ?

Comme il n'y a pas de réponse unique à ces questions, il n'y a pas d'approche unique de la tokenisaton. Il existe trois approches principales :

- Word-based : Découper le texte selon les espaces, et appliquer des règles spécifiques à la langue pour essayer de séparer des parties qui ont un sens propre même lorsqu'il n'y a pas d'espaces (comme transformer "don't" en "do not"). En général, les signes de ponctuation sont également divisés en tokens séparés.
- Subword-based : Découper les mots en plus petits morceaux, en vous basant sur les sous-chaînes les plus courantes. Par exemple, "préentrainement" peut être découpé en "pré|entrain|ement".
- Character-based : Découper une phrase en ses différents caractères.

### Tokenisation en mots avec fastai

Plutôt que de fournir son propre tokenizer, fastai fournit une interface cohérente avec des tokenizers fournis par des bibliothèques externes spécialisées. La tokenisation est un domaine de recherche actif, et des tokenizers nouveaux et améliorés sortent en permanence, de sorte que les paramètres par défaut utilisés par fastai changent également. Cependant, l'API et les options ne devraient pas trop changer, car fastai essaie de maintenir une API cohérente même si la technologie sous-jacente change.

In [2]:
from fastai2.text.all import *

In [3]:
path = untar_data(URLs.IMDB)

In [4]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [5]:
txt = files[0].open().read(); txt[:75]

"The worst movie I've ever seen, hands down. It is ten times more a rip-off "

In [6]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#156) ['The','worst','movie','I',"'ve",'ever','seen',',','hands','down','.','It','is','ten','times','more','a','rip','-','off','of','Lake','Placid','than','it','is','a','sequel','.','Director'...]


In [7]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [8]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#176) ['xxbos','xxmaj','the','worst','movie','xxmaj','i',"'ve",'ever','seen',',','hands','down','.','xxmaj','it','is','ten','times','more','a','rip','-','off','of','xxmaj','lake','xxmaj','placid','than','it'...]


Notez la création par le Tokenizer fastai de nouveaux **tokens spéciaux** qui commencent par les caractères "xx".

Par exemple, le premier élément de la liste, xxbos, est un token spécial qui indique le début d'un nouveau texte ("BOS" est un acronyme standard en NLP qui signifie "Beginning Of Stream"). En reconnaissant ce jeton de début, le modèle pourra apprendre qu'il doit "oublier" ce qui a été dit précédemment et se concentrer sur les mots à venir.

Ces tokens spéciaux ne proviennent pas directement de spaCy. Ils sont là parce que fastai les ajoute par défaut, en appliquant un certain nombre de règles lors du traitement du texte. Ces règles sont conçues pour permettre à un modèle de reconnaître plus facilement les parties importantes d'une phrase. Dans un sens, nous traduisons la séquence originale en anglais en un langage symbolique simplifié - un langage conçu pour être facile à apprendre pour un modèle.

Par exemple, les règles remplaceront une séquence de quatre points d'exclamation par un point d'exclamation unique, suivi d'un token spécial indiquant des caractères répétés, puis du chiffre quatre. De cette façon, la matrice d'embeddings du modèle peut coder des informations sur des concepts généraux tels que la ponctuation répétée plutôt que d'exiger un token séparé pour chaque nombre de répétitions de chaque signe de ponctuation.

De même, un mot en majuscules sera remplacé par un token spécial indiquant une majuscule, suivi de la version minuscule du mot. De cette façon, la matrice d'embeddings n'a besoin que des versions minuscules des mots, ce qui permet d'économiser des ressources de calcul et de mémoire, mais permet malgré tout d'apprendre le concept de capitalisation.

Voici quelques-uns des principaux tokens spéciaux que vous verrez :

xxbos: : Indique le début d'un texte (ici, une critique de films)
xxmaj: : Indique que le mot suivant commence par une majuscule (puisque nous avons tout mis en minuscule)
xxunk: : Indique que le mot suivant est inconnu (c'est à dire ne fait pas partie du vocabulaire de taille limitée)

Pour voir les règles qui ont été utilisées, vous pouvez vérifier les règles par défaut :

In [9]:
defaults.text_proc_rules

[<function fastai2.text.core.fix_html(x)>,
 <function fastai2.text.core.replace_rep(t)>,
 <function fastai2.text.core.replace_wrep(t)>,
 <function fastai2.text.core.spec_add_spaces(t)>,
 <function fastai2.text.core.rm_useless_spaces(t)>,
 <function fastai2.text.core.replace_all_caps(t)>,
 <function fastai2.text.core.replace_maj(t)>,
 <function fastai2.text.core.lowercase(t, add_bos=True, add_eos=False)>]

Voici un bref résumé de ce que chacune fait :

- fix_html : Remplace les caractères HTML spéciaux par une version lisible (les revues IMDb en ont plusieurs)
- replace_rep : Remplace tout caractère répété trois fois ou plus par un jeton spécial de répétition (xxrep), le nombre de fois qu'il est répété, puis le caractère
- replace_wrep: : Remplace tout mot répété trois fois ou plus par un jeton spécial pour la répétition des mots (xxwrep), le nombre de fois qu'il est répété, puis le mot
- spec_add_spaces : Ajoute des espaces autour de / et #
- rm_useless_spaces : Supprime toutes les répétitions du caractère espace
- replace_all_caps : Passe en minuscules les mots écrits tout en majuscules et ajoute un token spécial pour les mots tout en majuscules (xxcap)
- replace_maj : Passe en minuscules un mot commençant par une majuscule et ajoute un token spécial pour les mots qui commencent par une majuscule (xxmaj)
- minuscules : Passe tout le texte est en minuscules et ajoute un token spécial au début (xxbos) et/ou à la fin (xxeos)

Examinons quelques unes de ces transformations en action  :

In [52]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index'...]"

### Tokenisation en parties de mots avec fastai

La tokenisation en mots repose sur l'hypothèse que les espaces permettent une séparation utile des éléments de sens dans une phrase. 

Cependant, cette hypothèse n'est pas toujours appropriée. Prenons par exemple cette phrase : 我的名字是郝杰瑞 ("Mon nom est Jeremy Howard" en chinois). Cela ne fonctionnera pas très bien avec une tokenisation par mots, car elle ne contient pas d'espaces ! Des langues comme le chinois et le japonais n'utilisent pas d'espaces, et en fait, elles n'ont même pas de concept bien défini d'un "mot". Il existe également des langues, comme l'allemand, le turc et le hongrois, qui peuvent agglutiner de nombreux sous-mots sans espace, créant ainsi des mots très longs qui contiennent beaucoup d'informations séparées.

Pour traiter ces cas, il est généralement préférable d'utiliser une **tokenisation par sous-mots**. Cela se fait en deux étapes :

1. Analyser un corpus de documents pour trouver les groupes de lettres les plus courants. Ceux-ci deviennent le vocabulaire.
2. Tokeniser le corpus en utilisant ce vocabulaire constitué de sous-mots.

Voyons un exemple. Pour notre corpus, nous utiliserons les 2 000 premières critiques de films :

In [11]:
txts = L(o.open().read() for o in files[:2000])

Nous instancions notre tokenizer, en lui passant la taille du vocabulaire que nous voulons créer, et ensuite nous devons l'"entrainer". C'est-à-dire que nous devons lui faire lire nos documents et trouver les séquences de caractères les plus fréquentes pour créer le vocabulaire. Cela est fait dans la fonction setup().

In [12]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [13]:
subword(1000)

"▁The ▁worst ▁movie ▁I ' ve ▁ever ▁seen , ▁hand s ▁down . ▁It ▁is ▁t en ▁time s ▁more ▁a ▁ r i p - off ▁of ▁L ake ▁P la ci d ▁than ▁it ▁is ▁a ▁sequel ."

Lorsque vous utilisez le tokenizer en sous-mots de fastai, le caractère spécial ▁ représente un espace dans le texte original.

Si nous utilisons un vocabulaire plus petit, alors chaque token représentera moins de caractères, et il faudra plus de tokens pour représenter une phrase :

In [14]:
subword(200)

"▁The ▁w or s t ▁movie ▁I ' ve ▁ e ver ▁s e en , ▁ h an d s ▁d o w n . ▁I t ▁is ▁ t en ▁ t i m es ▁mo re ▁a"

In [15]:
subword(10000)

"▁The ▁worst ▁movie ▁I ' ve ▁ever ▁seen , ▁hands ▁down . ▁It ▁is ▁ten ▁times ▁more ▁a ▁rip - off ▁of ▁Lake ▁Placid ▁than ▁it ▁is ▁a ▁sequel . ▁Director ▁David ▁F lo re s ▁clearly ▁did ▁not ▁go"

Le choix de la taille du vocabulaire des sous-mots représente un compromis : un vocabulaire plus grand signifie moins de tokens par phrase, ce qui signifie un entraînement plus rapide, moins de mémoire et moins d'états à mémoriser pour le modèle ; mais à l'inverse, cela signifie des matrices d'embeddings plus grandes, qui nécessitent plus de données pour l'apprentissage.

Dans l'ensemble, la tokenisation en sous-mots permet d'ajuster un bon compromis entre la tokenisation par caractères (c'est-à-dire l'utilisation d'un tout petit vocabulaire de sous-mots) à la tokenisation par mots (c'est-à-dire l'utilisation d'un très grand vocabulaire de sous-mots), et traite toutes les langues humaines sans qu'il soit nécessaire de développer des algorithmes spécifiques à la langue. 

Il peut même gérer d'autres "langues" telles que les séquences génomiques ou la notation musicale MIDI ! C'est pourquoi, l'année dernière, sa popularité a explosé et il devient l'approche la plus courante en matière de tokenisation.

### Conversion en numéros avec fastai

La "numérisation" est le processus de mise en correspondance de tokens avec des entiers. Les étapes sont fondamentalement identiques à celles nécessaires pour créer une variable de catégorie :

1. Faire une liste de toutes les valeurs possibles dans cette variable de type catégorie (le vocabulaire).
2. Remplacez chaque niveau par son index dans le vocabulaire.

In [53]:
toks = tkn(txt)
print(coll_repr(toks, 31))

(#176) ['xxbos','xxmaj','the','worst','movie','xxmaj','i',"'ve",'ever','seen',',','hands','down','.','xxmaj','it','is','ten','times','more','a','rip','-','off','of','xxmaj','lake','xxmaj','placid','than','it'...]


In [17]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#176) ['xxbos','xxmaj','the','worst','movie','xxmaj','i',"'ve",'ever','seen'...]

In [18]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2144) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','in','it','i'...]"

Nos tokens spéciaux apparaissent en premier, puis chaque mot apparaît une fois, par ordre de fréquence décroissant.

Les paramètres par défaut de Numericalize sont min_freq=3, max_vocab=60 000. max_vocab=60 000 entraîne le remplacement par fastai de tous les mots autres que les 60 000 les plus courants par un token spécial de mots inconnus, xxunk. Ceci est utile pour éviter d'avoir une matrice d'embeddingfs trop grande, car cela peut ralentir l'entraînement et utiliser trop de mémoire, et peut également signifier qu'il n'y a pas assez de données pour entraîner des représentations utiles pour les mots rares. Cependant, ce dernier problème est mieux géré en définissant min_freq ; la valeur par défaut min_freq=3 signifie que tout mot apparaissant moins de trois fois est remplacé par xxunk.

fastai peut également convertir en nombres votre jeu de données en utilisant un vocabulaire que vous fournissez, en passant une liste de mots dans le paramètre vocab.

In [19]:
nums = num(toks)[:20]; nums

tensor([  2,   8,   9, 310,  27,   8,  19, 218, 158, 141,  10,   0, 229,  11,
          8,  18,  16, 550, 299,  66])

In [20]:
' '.join(num.vocab[o] for o in nums)

"xxbos xxmaj the worst movie xxmaj i 've ever seen , xxunk down . xxmaj it is ten times more"

### Agréger nos textes en batchs pour un Language Model

Pour traiter des images, nous devions les redimensionner toutes à la même hauteur et largeur avant de les regrouper en un mini-batch afin qu'elles puissent être groupées efficacement dans un seul tenseur. Ici, ce sera un peu différent, car on ne peut pas simplement redimensionner le texte à la longueur souhaitée. Nous voulons également que notre modèle linguistique lise le texte dans l'ordre, afin de pouvoir prédire efficacement le mot suivant. Cela signifie que chaque nouveau batch doit commencer exactement là où le précédent s'est arrêté.

Supposons que nous avons 90 tokens. Disons que nous voulons une taille de batch de 6, chaque élement de batch étant une séquence de tokens. Nous devons décomposer ce texte en 6 parties contiguës de longueur 15 :

In [21]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
1,movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
2,first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
3,how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
4,of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
5,will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


Dans un monde parfait, nous pourrions passer directement ce batch à notre modèle. Mais cette approche ne passe pas à l'échelle, car en dehors de cet exemple simpliste, il est peu probable qu'un batch contenant tous les textes puisse tenir dans notre mémoire GPU (ici, nous avons 90 tokens, mais toutes les critiques IMDb réunies en contiennent plusieurs millions).

Nous devons donc diviser ce tableau plus finement en sous-tableaux d'une longueur de séquence fixe. Il est important de maintenir l'ordre au sein de ces sous-tableaux et entre eux, car nous utiliserons un modèle qui maintient un état de sorte qu'il se souvienne de ce qu'il a lu précédemment lorsqu'il prédit ce qui va suivre.

Pour revenir à notre exemple précédent avec 6 lots de longueur 15, si nous choisissions une longueur de séquence de 5, cela signifierait que nous alimenterions d'abord le GPU avec le tableau suivant :

In [22]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
df

,0,1,2,3,4
0,xxbos,xxmaj,in,this,chapter
1,movie,reviews,we,studied,in
2,first,we,will,look,at
3,how,to,customize,it,.
4,of,the,preprocessor,used,in
5,will,study,how,we,build


Puis avec celui-là :

In [23]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
df

,0,1,2,3,4
0,",",we,will,go,back
1,chapter,1,and,dig,deeper
2,the,processing,steps,necessary,to
3,xxmaj,by,doing,this,","
4,the,data,block,xxup,api
5,a,language,model,and,train


Etc ... et finalement celui-là :

In [24]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
df

,0,1,2,3,4
0,over,the,example,of,classifying
1,under,the,surface,.,xxmaj
2,convert,text,into,numbers,and
3,we,'ll,have,another,example
4,.,\n,xxmaj,then,we
5,it,for,a,while,.


Pour en revenir à notre base de données de critiques de films, la première étape consiste à transformer les textes individuels en un flux en les concaténant tous ensemble. Comme pour les images, il est préférable de faire varier l'ordre des inputs en cours d'entrainement, ainsi au début de chaque époque nous allons mélanger les inputs pour créer un nouveau flux (nous mélangeons l'ordre des documents, pas l'ordre des mots à l'intérieur de ceux-ci, ou les textes n'auraient plus de sens !).

Nous découpons ensuite ce flux en un certain nombre de batchs (ce qui correspond à notre taille de batch). Par exemple, si le flux comporte 50 000 tokens et que nous fixons une taille de batch de 10, cela nous donnera 10 mini-flux de 5 000 tokens. L'important est de conserver l'ordre des tokens (donc de 1 à 5 000 pour le premier mini-stream, puis de 5 001 à 10 000...), car nous voulons que le modèle lise des lignes de texte continues (comme dans l'exemple précédent). Un jeton xxbos est ajouté au début de chacune d'entre elles lors du prétraitement, afin que le modèle sache quand il lit le flux lorsqu'une nouvelle entrée commence.

Pour résumer, à chaque époque, nous remanions notre collection de documents et les concaténons en un flux de tokens. Nous découpons ensuite ce flux en un batch de mini-flux consécutifs de taille fixe. Notre modèle lira alors les mini-streams dans l'ordre, et grâce à un état interne, il produira la même activation quelle que soit la longueur de la séquence que nous aurons choisie.

Tout cela est réalisé en coulisse par la bibliothèque fastai lorsque nous créons un LMDataLoader.

In [54]:
nums200 = toks200.map(num)

In [26]:
dl = LMDataLoader(nums200)

 Paramètres par défaut de LMDataLoader : bs=64, seq_len=72.

In [27]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [28]:
' '.join(num.vocab[o] for o in x[0][:20])

"xxbos xxmaj the worst movie xxmaj i 've ever seen , xxunk down . xxmaj it is ten times more"

In [29]:
' '.join(num.vocab[o] for o in y[0][:20])

"xxmaj the worst movie xxmaj i 've ever seen , xxunk down . xxmaj it is ten times more a"

## Entrainer un classifier de texte

### Utilisation de DataBlock pour un Language Model 

fastai gère la tokenisation et la numérisation automatiquement lorsqu'un TextBlock est passé au DataBlock. Tous les arguments qui peuvent être passés à Tokenize et Numericalize peuvent également être passés à TextBlock.

In [40]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

Une chose qui diffère des types précédents que nous avons utilisés dans DataBlock est que nous n'utilisons pas seulement la classe directement (c'est-à-dire TextBlock(...), mais que nous appelons plutôt une méthode de classe. Une méthode de classe est une méthode Python qui, comme son nom l'indique, appartient à une classe plutôt qu'à un objet.

La raison pour laquelle TextBlock est spécial est que la mise en place du vocabulaire et de la conversion en nombres peut prendre beaucoup de temps (nous devons lire et tokeniser tous les documents pour obtenir le vocabulaire). Pour être le plus efficace possible, il faut procéder à quelques optimisations :

1. Il enregistre les documents tokenisés dans un dossier temporaire, afin de ne pas avoir à les tokeniser plus d'une fois
2. Il exécute plusieurs processus de tokenisation en parallèle, pour tirer parti des CPUs de votre ordinateur

Nous devons indiquer à TextBlock comment accéder aux textes, afin qu'il puisse effectuer ce prétraitement initial - c'est ce que fait from_folder.

In [41]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj if you want to laugh like crazy , rent xxmaj cage . xxmaj cage is about two war heroes , xxmaj billy and xxmaj scott who are best friends . xxmaj when xxmaj billy is shot in xxmaj vietnam , he is unable to fend for himself , so xxmaj scott takes him in . \n\n i have never seen a movie with more gay references to the two main characters . xxmaj billy and xxmaj scott love","xxmaj if you want to laugh like crazy , rent xxmaj cage . xxmaj cage is about two war heroes , xxmaj billy and xxmaj scott who are best friends . xxmaj when xxmaj billy is shot in xxmaj vietnam , he is unable to fend for himself , so xxmaj scott takes him in . \n\n i have never seen a movie with more gay references to the two main characters . xxmaj billy and xxmaj scott love to"
1,", the characters were boring , and i did n't believe in their relationship for more than a few minutes . \n\n xxmaj furthermore , and xxup worst of all , i was very confused how he died simply diving into a body of water , and was really irritated that they did n't explain that in the movie . xxmaj it was only after i did some research on - line and read other reviews , that someone said","the characters were boring , and i did n't believe in their relationship for more than a few minutes . \n\n xxmaj furthermore , and xxup worst of all , i was very confused how he died simply diving into a body of water , and was really irritated that they did n't explain that in the movie . xxmaj it was only after i did some research on - line and read other reviews , that someone said it"


### Fine-Tuning du Language Model

Attention, dans Gradient, le répertoire de stockage des modèles par défaut est en lecture seule.

On spécifie donc explicitement le répertoire de sauvegarde de notre modèle :

In [42]:
Config().model

Path('/storage/models')

In [65]:
rw_model_path = Path("~/.fastai").expanduser()
if not rw_model_path.exists():
    rw_model_path.mkdir(parents=True)

Pour convertir les indices de mots en activations que nous pouvons utiliser pour notre réseau neuronal, nous utiliserons des embeddings, tout comme nous l'avons fait pour le filtrage collaboratif et la modélisation de données structurées.

Ensuite, nous fournissons ces embeddings en entrée d'un **réseau de neurones récurrent** (RNN), en utilisant une architecture appelée AWD-LSTM.

In [44]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()],
    path=rw_model_path).to_fp16()

La fonction de coût utilisée par défaut est cross entropy, car nous traitons un problème de classification (les différentes catégories étant les mots de notre vocabulaire). 

La métrique de perplexité utilisée ici est souvent utilisée en NLP pour les modèles de langue : c'est l'exponentielle de la fonction de coût (c'est-à-dire torch.exp(cross_entropy)). 

Nous incluons également la mesure de la précision, pour voir combien de fois notre modèle est correct lorsque nous essayons de prédire le mot suivant, car la cross entropy est à la fois difficile à interpréter et nous en dit plus sur la confiance du modèle que sur sa précision.

Attention, l'entrainement ci-dessous dure 40 min avec une machine P5000 sous Gradient :

In [45]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.127624,3.918828,0.299556,50.341396,38:36


### Enregistrer et recharger des modèles

In [46]:
learn.save('1epoch') # 370 Mo

Cela créera un fichier dans learn.path/models/ nommé 1epoch.pth. Si vous souhaitez charger votre modèle sur une autre machine après avoir créé votre Learner de la même manière, ou reprendre l'entrainement plus tard, vous pouvez charger le contenu de ce fichier avec :

In [47]:
learn = learn.load('1epoch')

Attention, l'entrainement ci-dessous dure 6 heures sur une machine P5000 sous Gradient :

In [48]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

KeyboardInterrupt: 

Une fois cela fait, nous sauvegardons tout notre modèle sauf la couche finale qui convertit les activations en probabilités de choisir chaque token de notre vocabulaire. Le modèle n'incluant pas la couche finale est appelé l'encodeur. Nous pouvons le sauvegarder avec save_encoder :

Encodeur : Le modèle n'incluant pas la ou les couches finales spécifiques à la tâche. Ce terme signifie à peu près la même chose que body lorsqu'il est appliqué aux CNN de vision, mais "encodeur" tend à être plus utilisé pour le NLP et les modèles de génération de séquences.

In [49]:
learn.save_encoder('finetuned') # 177 Mo

In [71]:
(learn.path / "models").ls()

(#2) [Path('/root/.fastai/models/1epoch.pth'),Path('/root/.fastai/models/finetuned.pth')]

### Génération de texte

In [50]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [51]:
print("\n".join(preds))

i liked this movie because it had any sexual content i thought . In the first half of the movie ( which i also gave of the marketing ) , i was almost relieved when i saw the trailer . i think this one
i liked this movie because of the great cast and the fantastic special effects . i am not a big fan of Eugene Palette but i was a huge fan of the early Hitchcock films . In the day , i


Notre modèle n'a aucune connaissance programmée de la structure d'une phrase ou des règles de grammaire, mais il a clairement appris beaucoup de choses sur les phrases anglaises : nous pouvons voir qu'il met correctement les majuscules (I est transformé en i parce que nos règles exigent deux caractères ou plus pour considérer un mot comme commençant par un majuscule, il est donc normal de le voir en minuscules) et il applique une concordance des temps. 

### Créer les DataLoaders pour le classifier

In [72]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [73]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * * attention xxmaj spoilers * * \n\n xxmaj first of all , let me say that xxmaj rob xxmaj roy is one of the best films of the 90 's . xxmaj it was an amazing achievement for all those involved , especially the acting of xxmaj liam xxmaj neeson , xxmaj jessica xxmaj lange , xxmaj john xxmaj hurt , xxmaj brian xxmaj cox , and xxmaj tim xxmaj roth . xxmaj michael xxmaj canton xxmaj jones painted a wonderful portrait of the honor and dishonor that men can represent in themselves . xxmaj but alas … \n\n it constantly , and unfairly gets compared to "" braveheart "" . xxmaj these are two entirely different films , probably only similar in the fact that they are both about xxmaj scots in historical xxmaj scotland . xxmaj yet , this comparison frequently bothers me because it seems",pos
2,"xxbos xxmaj some have praised xxunk xxmaj lost xxmaj xxunk as a xxmaj disney adventure for adults . i do n't think so -- at least not for thinking adults . \n\n xxmaj this script suggests a beginning as a live - action movie , that struck someone as the type of crap you can not sell to adults anymore . xxmaj the "" crack staff "" of many older adventure movies has been done well before , ( think xxmaj the xxmaj dirty xxmaj dozen ) but xxunk represents one of the worse films in that motif . xxmaj the characters are weak . xxmaj even the background that each member trots out seems stock and awkward at best . xxmaj an xxup md / xxmaj medicine xxmaj man , a tomboy mechanic whose father always wanted sons , if we have not at least seen these before ,",neg


Si l'on regarde la définition du DataBlock, chaque élément est familier, à deux exceptions près :
- TextBlock.from_folder n'a plus le paramètre is_lm=True.
- Nous passons le vocabulaire que nous avons créé pour la mise au point du modèle de langue.

La raison pour laquelle nous transmettons le vocabulaire du modèle de langue est de nous assurer que nous utilisons la même correspondance avec les tokens pour l'indexation. Sinon, les embeddings que nous avons appris dans notre modèle de langue affiné n'auront aucun sens pour ce modèle, et l'étape de fine tuning n'aura été d'aucune utilité.

En passant is_lm=False (ou en ne passant pas is_lm du tout, puisque la valeur par défaut est False), nous disons à TextBlock que nous avons des données avec des labels explicites, plutôt que d'utiliser les tokens suivants comme labels. 

Il y a cependant un défi à relever, qui consiste à rassembler plusieurs documents en un mini batch. Voyons un exemple, en essayant de créer un mini batch contenant les 10 premiers documents. Nous allons d'abord les numériser :

In [81]:
nums_samp = toks200[:10].map(num)

Nombre de tokens pour chacun de ces dix documents : chaque document a une longueur différente.

In [82]:
nums_samp.map(len)

(#10) [176,339,205,779,123,916,893,196,106,170]

Rappelez-vous, les DataLoaders PyTorch doivent rassembler tous les éléments d'un batch en un seul tenseur, et un tenseur a une forme fixe (c'est-à-dire qu'il a une longueur particulière sur chaque axe, et tous les éléments doivent être cohérents). 

Cela doit vous sembler familier : nous avons eu le même problème avec les images. Dans ce cas, nous avons utilisé le recadrage, le padding et/ou le cropping pour que toutes les entrées aient la même taille. Le recadrage n'est peut-être pas une bonne idée pour les documents textuels, car il semble probable que nous enlèverions certaines informations clés. Il reste donc le padding.

Nous allons agrandir les textes les plus courts pour qu'ils aient tous la même taille. Pour ce faire, nous utilisons un token de padding spécial qui sera ignoré par notre modèle. 

De plus, pour éviter les problèmes de mémoire et améliorer les performances, nous regrouperons les textes qui sont à peu près de la même longueur (avec un certain remaniement pour le jeu d'entraînement). Pour ce faire, nous trions (approximativement, pour le jeu d'entraînement) les documents par longueur avant chaque époque. Le résultat est que les documents rassemblés dans un batch auront tendance à être de longueur similaire. 

Nous n'allons pas appliquer un padding à la même taille pour tous les batchs, mais nous utiliserons plutôt la taille du plus grand document de chaque batch comme taille cible.

Le tri et le padding sont automatiquement effectués par l'API DataBlock pour nous lorsque nous utilisons un TextBlock, avec is_lm=False. (Nous n'avons pas ce même problème pour les données des modèles de langue, puisque nous concaténons d'abord tous les documents ensemble, puis nous les divisons en sections de taille égale).

In [78]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy,
                                path=rw_model_path).to_fp16()

In [79]:
learn = learn.load_encoder('finetuned')

### Fine-Tuning du classifier

La dernière étape consiste à entraîner avec "discriminative learning rates" et "progressive unfreezing". 

En vision par ordinateur, nous appliquons souvent "unfreeze()" sur le modèle d'un seul coup, mais pour les classifiers en NPL, nous constatons que le unfreezing couche par couhe fait une réelle différence.

In [80]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.381519,0.243214,0.904440,02:02


Une fois qu'on a entrainé un language model, le fine tuning d'un classifier est beaucoup plus rapide : 2 minutes par époque.

In [83]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.284521,0.211871,0.916400,02:19


In [84]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.221611,0.195495,0.925120,03:12


In [85]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.200107,0.189627,0.928280,03:50
1,0.185894,0.183830,0.930280,03:49


Nous avons atteint une précision de 94,3 %, ce qui correspond à l'état de l'art il y a seulement trois ans. En entrainant un autre modèle sur tous les textes lus à l'envers et en faisant la moyenne des prédictions de ces deux modèles, nous pouvons même atteindre une précision de 95,1%, ce qui était l'état de l'art introduit par le papier ULMFiT. Il n'a été battu qu'il y a quelques mois, en affinant un modèle beaucoup plus grand et en utilisant des techniques coûteuses d'augmentation des données (traduction de phrases dans une autre langue et retour, utilisation d'un autre modèle pour la traduction).

L'utilisation d'un modèle pré-entrainé nous a permis de construire un modèle de langue affiné qui était assez puissant, pour générer de fausses critiques ou aider à les classer. C'est passionnant, mais il est bon de se rappeler que cette technologie peut aussi être utilisée à des fins malveillantes.

## Désinformation et Language Models

https://openai.com/blog/better-language-models/